# Instance Based Learning

In this notebook you will learn how to implement the k-Nearest Neighbors (kNN) algorithm in Python and learn how to
use kNN and SVN algorithms in scikit-learn.

## The Dataset

In this notebook you will be working with the [Wine dataset](https://archive.ics.uci.edu/ml/datasets/Wine).
This dataset is the result of a chemical analysis of wines
grown in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents
found in each of the three types of wines.

This is a good dataset for testing classifiers. In the following table we find some properties of this dataset.

|Property|Value|
|--|--|
|Classes|3|
|Samples per class|~59|
|Samples total|178|
|Dimensionality|13|
|Features|positive, natural and real|

Each example contains a class identifier and 13 attributes representing the outcome of the analysis performed on the
wine samples.

We will download this dataset directly from the UCI repository using Pandas. Note that this dataset does not have a
header which means that we need to provide the column names manually. This header comes from reading the content of this
[file](https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.names)
(you can open this file with a text editor).

In [ ]:
import pandas as pd

dataset_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

names = ['class', # label
         'alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total phenols', 'flavanoids',
         'non-flavanoid phenols', 'proanthocyanins', 'color_intensity', 'hue', 'protein_content', 'proline']

df = pd.read_csv(dataset_url, names=names)

Let's first have a look at the content of this dataframe.

In [ ]:
df.head()

Let's also have a look at some statistics of the dataframe.

In [ ]:
df.describe()

## Preprocessing the Dataset

This dataset will require some preprocessing. First, since we would like to test how any of the learning algorithms
used later perform on unseen data. We split the dataset into a training set and a test set.

To do this we first randomize the data. We do this in order to make sure to select an unbiased set of examples
for the test set. Notice that in this case the training is sorted based on the target `class`, which makes the
initial randomization necessary.

In [ ]:
df = df.sample(frac=1)
df.head()

We now separate the target `class` from the rest of the attributes.

In [ ]:
import numpy as np

# this sets the numpy to print numbers with float precision (this setting affects only the prints not the actual values)
np.set_printoptions(suppress=True)

ys, xs = np.split(df.values, [1], axis=1)
ys = ys.reshape(-1)

And select 80% of the dataset for training and 20% for testing.

In [ ]:
n_train = len(xs) * 80 // 100
xs_train, xs_test = np.split(xs, [n_train], axis=0)
ys_train, ys_test = np.split(ys, [n_train], axis=0)

print('training set shape:\t', xs_train.shape)
print('test set shape:\t\t', xs_test.shape)

Now, we note that attribute values span across various ranges: Some attributes have a much wider range than others.
Since the learning algorithms that we will be using later are based on some form of distance function,
this variance in the ranges of the attributes may bias the learning algorithm towards examples that look closer in
dimensions with a wider range because those attributes will dominate the distance functions.
In order to mitigate this bias we perform a normalization across the features. In this case we will perform a
standardization (aka Z-score):

\begin{equation}
z = \frac{x-\mu}{\sigma}
\end{equation}

Any other normalization that achieves a similar result is also fine.

In [ ]:
mu = np.mean(xs_train, axis=0)
sigma = np.std(xs_train, axis=0)

xs_train = (xs_train - mu)/sigma
xs_test = (xs_test - mu)/sigma

Note that the normalization should be computed on the training set and not on the original dataset. This in order to
better simulate unseen data. The normalization, together with any preprocessing step that involves statistics over
the dataset, should also be considered as belonging to the hyper-parameters of the learning algorithm.

After having performed the normalization, if we now compute the mean of the preprocessed training set, we should see
that now this mean vector contains only zeros.

In [ ]:
np.mean(xs_train, axis=0)

If we have properly sampled the dataset, we should get a mean vector for the test set that contains close to zero
values.

In [ ]:
np.mean(xs_test, axis=0)


## The Nearest Neighbor Algorithm

We will now implement the Nearest Neighbor (NN) algorithm in Python from scratch.

In [ ]:
class NN:

    def __init__(self, distance):
        self.training_examples = []
        self.distance = distance

    def add_example(self, x, y):
        """
        Add one example to the list of training examples.
        :param x: The vector of feature values
        :param y: The label associated to this example
        """
        self.training_examples.append((x, y))

    def add_examples(self, xs, ys):
        """
        Add a list of examples to the list of training examples.
        :param xs: A list of vectors of fature values
        :param ys: A list of labels associated to the examples
        """
        for x, y in zip(xs, ys):
            self.add_example(x, y)

    def closest_training_example(self, x_q):
        y_closest = None
        x_closest = None
        min_score = float('inf')
        # find closest example
        for x, y in self.training_examples:
            score = self.distance(x_q, x)
            if score < min_score:
                min_score = score
                x_closest = x
                y_closest = y

        return x_closest, y_closest

    def classify(self, xq):
        _, y_hat = self.closest_training_example(xq)
        return y_hat

In order to instantiate this classifier we need to define a distance function. Since we are dealing with continuous
features we will define the euclidean distance. You are invited to develop and test another distance measure.

In [ ]:
def euclidean_distance(x_1, x_2):
    res = 0
    for a_1, a_2 in zip(x_1, x_2):
        res += (a_1 - a_2) ** 2
    res **= 0.5
    return res

The euclidean distance of the points (0, 0) and (1, 1) should be $\sqrt{2}$.

In [ ]:
euclidean_distance([0, 0], [1, 1])

We now instantiate the NN classifier and train it.

In [ ]:
nn_clf = NN(euclidean_distance)

nn_clf.add_examples(xs_train, ys_train)

To evaluate how this classifier performs on the test set we will measure its accuracy. Note that evaluating the
accuracy on the training set is pointless because this will always be 1 by definition. We will now do also this only for
instructive purposes.

We now define the accuracy measure. Remember that the accuracy is equal to the proportion of examples that the
classifier predicted correctly.

In [ ]:
def accuracy(ys, ys_hat):
    res = 0
    for y, y_hat in zip(ys, ys_hat):
        if y == y_hat:
            res += 1
    res /= len(ys)
    return res

We now test the classifier on both training and test sets.

In [ ]:
ys_train_pred = []
for x in xs_train:
    y_hat = nn_clf.classify(x)
    ys_train_pred.append(y_hat)

ys_test_pred = []
for x in xs_test:
    y_hat = nn_clf.classify(x)
    ys_test_pred.append(y_hat)

print('Train accuracy of NN', accuracy(ys_train, ys_train_pred))
print('Test accuracy of NN', accuracy(ys_test, ys_test_pred))

Let's now compare the test result of this classifier to a random classifier. Is this NN classifier any better?
Remember that a random classifier would correctly predict the class one third of the times.

In [ ]:
ys_test_pred_random = np.random.randint(1, 4, len(ys_test))
print('Test accuracy of a random classifier', accuracy(ys_test, ys_test_pred_random))

Let's have a look at the result of a random classifier by repeating this experiment many times.

In [ ]:
from matplotlib import pyplot as plt

accuracies = []
for _ in range(10000):
    ys_test_pred_random = np.random.randint(1, 4, len(ys_test))
    accuracies.append(accuracy(ys_test, ys_test_pred_random))

plt.hist(accuracies)
plt.show()

print('Expected accuracy of a random classifier', np.mean(accuracies))

Does this accuracy make sense? Is this accuracy similar to the one you had predicted?

# The k-Nearest Neighbor Algorithm

We will now extend the NN algorithm to develop the k-Nearest Neighbor (kNN) algorithm.
By extending the NN algorithm we avoid repeating the training code. This is in fact the same.
In the classification method, in order to avoid the sorting of
all the examples after having measured their score, we will make use of priority queues, which allow us to keep track
only of the first $k$-nearest examples, making the code more efficient. You are free to implement the version where
first all the examples are scored, then sorted and selected. These two versions, if correctly implemented,
should produce to the same result.

In [ ]:
from statistics import mode
from heapq import heappush, heappushpop

class KNN(NN):

    def __init__(self, distance):
        super().__init__(distance)

    def closest_training_examples(self, x_q, k=1):
        k_nearest = []

        # initialize an heap with k elements
        for x, y  in self.training_examples[:k]:
            score = self.distance(x_q, x)
            heappush(k_nearest, (-score, (x, y)))

        # find the k-nearest example
        for x, y in self.training_examples[k:]:
            score = self.distance(x_q, x)
            heappushpop(k_nearest, (-score, (x, y)))

        # we no longer need to keep the score
        res = [(x, y) for _, (x, y) in k_nearest]
        return res

    def classify(self, x_q, k = 1):
        # find the k closest
        k_nearest_xs, k_nearest_ys = zip(*self.closest_training_examples(x_q, k))
        # return the mode
        return mode(k_nearest_ys)

We now train and test this algorithm in the same way we did for the NN algorithm.

In [ ]:
knn_clf = KNN(euclidean_distance)

knn_clf.add_examples(xs_train, ys_train)

ys_train_pred = []
for x in xs_train:
    y_hat = knn_clf.classify(x)
    ys_train_pred.append(y_hat)

ys_test_pred = []
for x in xs_test:
    y_hat = knn_clf.classify(x)
    ys_test_pred.append(y_hat)

print('Train accuracy of kNN', accuracy(ys_train, ys_train_pred))
print('Test accuracy of kNN', accuracy(ys_test, ys_test_pred))

Nothing has changed with respect to the NN algorithm because we have implicitly used $k=1$. Let's now try $k=5$.

In [ ]:
ys_train_pred = []
for x in xs_train:
    y_hat = knn_clf.classify(x, 5)
    ys_train_pred.append(y_hat)

ys_test_pred = []
for x in xs_test:
    y_hat = knn_clf.classify(x, 5)
    ys_test_pred.append(y_hat)


print('Train accuracy of kNN', accuracy(ys_train, ys_train_pred))
print('Test accuracy of kNN', accuracy(ys_test, ys_test_pred))

It seems that considering more points did not help. Note that this time the training accuracy has changed. Can you
explain why?

## kNN in Scikit-Learn

We will now learn how to use the kNN implementation of scikit-learn.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=5, metric='euclidean')

To train the classifier.

In [ ]:
knn_clf.fit(xs_train, ys_train)

We now evaluate the result of this classifier. Here, we should not see any
difference with respect to the results obtained above with our implementation of the kNN algorithm.

In [ ]:
from sklearn.metrics import accuracy_score

ys_test_pred = knn_clf.predict(xs_test)

print('Test accuracy of kNN', accuracy_score(ys_test, ys_test_pred))

Let's now try the cosine distance.

In [ ]:
knn_clf = KNeighborsClassifier(n_neighbors=1, metric='cosine')

knn_clf.fit(xs_train, ys_train)

ys_test_pred = knn_clf.predict(xs_test)

print('Test accuracy of kNN', accuracy_score(ys_test, ys_test_pred))

Try other $n$ values. Can you find a better one? The danger of doing this hyper-parameter exploration using the test set is
that we may overfit these hyper-parameters on the test set.
To avoid this, it is better to find the best hyper-parameter values via a validation strategy.

To find these hyper-parameter values we can exploit the grid search of scikit-learn. This will perform a k-fold
cross-validation on the training set.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [{
    'weights': ["uniform", "distance"],
    'n_neighbors': range(1, 11),
    'metric':['euclidean', 'manhattan', 'cosine']}]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=2)
grid_search.fit(xs_train, ys_train)

We can now see what are the best hyper-parameter values found by the cross-validation.

In [ ]:
grid_search.best_estimator_

Let's now try this hyper-parameters on the test set.

In [ ]:
knn_clf = KNeighborsClassifier(metric='cosine', n_neighbors=4, weights='distance')

knn_clf.fit(xs_train, ys_train)

ys_train_pred = knn_clf.predict(xs_train)
ys_test_pred = knn_clf.predict(xs_test)

print('Train accuracy of kNN', accuracy(ys_train, ys_train_pred))
print('Test accuracy of kNN', accuracy(ys_test, ys_test_pred))

The accuracy measured on the test set is now a better estimate of the accuracy we would expect on unseen examples.

# Support Vector Machines

Here I will introduce you how to use the Support Vector Machine (SVM) implementation of scikit-learn.

Note how we are setting the $C$ hyper-parameter of SVM. $C$ controls the trade-off between having a small and strict
margin and a wider and loose margin. Following we will set $C$ to infinity which makes the margin infinitely strict.
This means that based on the dataset, the fitting of the SVM may fail if the training algorithm fails to separate all
the training examples perfectly.

In [ ]:
from sklearn.svm import SVC

svm_clf = SVC(kernel="linear", C=float("inf"))

We now train this classifier.

In [ ]:
svm_clf.fit(xs_train, ys_train)

The training went well, which means that the SVM training algorithm managed to perfectly fit the training examples.
We can now verify this on the training set. We will also test the performance of this classifier on the test set.

In [ ]:
ys_train_pred = svm_clf.predict(xs_train)
ys_test_pred = svm_clf.predict(xs_test)

print('Train accuracy of SVM', accuracy(ys_train, ys_train_pred))
print('Test accuracy of SVM', accuracy(ys_test, ys_test_pred))

How would you find the best hyper-parameter C value? Try re-implement the code used to validate the kNN
classifier above.